In [1]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
# Req for package
sys.path.append("../")
from src.Dataset.Utils import getDataset, getSplit
from src.NN.Utils import train, test, DEVICE

torch.backends.cudnn.benchmark = True

# Model

In [2]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.fc1 = nn.Linear(2176, 512)
        self.d1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(512, 256)
        self.d2 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(256, 64)
        self.d3 = nn.Dropout(0.5)
        
        self.fc4 = nn.Linear(64, 6)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(-1, 2, 128, 1)

        x = self.pool1(torch.relu(self.conv1(x)))
        """x = x.reshape(batch_size, 32, 1, -1)
        x = self.bn1(x)
        x = x.reshape(batch_size, 32, -1)"""
        
        x = self.pool2(torch.relu(self.conv2(x)))
        """x = x.reshape(batch_size, 64, 1, -1)
        x = self.bn2(x)
        x = x.reshape(batch_size, 64, -1)"""
        
        x = self.pool3(torch.relu(self.conv3(x)))
        """ x = x.reshape(batch_size, 128, 1, -1)
        x = self.bn3(x)
        x = x.reshape(batch_size, 128, -1)"""
        
        # = self.pool4(torch.relu(self.conv4(x)))
        
        x = x.view(batch_size, -1)
        x = self.d1(torch.relu(self.fc1(x)))
        x = self.d2(torch.relu(self.fc2(x)))
        x = self.d3(torch.relu(self.fc3(x)))
        x = self.fc4(x)
        x = x.reshape(batch_size, -1)
        return x


# Dataset

In [3]:
dataset, scaler = getDataset()

100%|██████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:38<00:00, 58.36it/s]


## Train/test split

In [4]:
train_loader, test_loader = getSplit(dataset)

# Training/testing

In [5]:
net = MyCNN()

In [6]:
train(train_loader, net, val_loader=test_loader, LR=0.0001)

Using: cuda:0


Epoch 68/2000: 100%|█████████████████████████████████| 56/56 [00:02<00:00, 26.72batch/s, lastLoss=0.166, valLoss=0.168]


Epoch 136/2000: 100%|████████████████████████████████| 56/56 [00:02<00:00, 19.33batch/s, lastLoss=0.137, valLoss=0.133]


Epoch 204/2000: 100%|████████████████████████████████| 56/56 [00:02<00:00, 20.09batch/s, lastLoss=0.129, valLoss=0.114]


Epoch 272/2000: 100%|████████████████████████████████| 56/56 [00:02<00:00, 19.93batch/s, lastLoss=0.126, valLoss=0.113]


...

Epoch 1904/2000: 100%|██████████████████████████████| 56/56 [00:01<00:00, 28.98batch/s, lastLoss=0.108, valLoss=0.0929]


Epoch 1972/2000: 100%|██████████████████████████████| 56/56 [00:01<00:00, 28.41batch/s, lastLoss=0.107, valLoss=0.0961]


Epoch 2000/2000: 100%|██████████████████████████████| 56/56 [00:01<00:00, 29.00batch/s, lastLoss=0.107, valLoss=0.0975]


Parameters: Skin YM, Adipose YM, Skin PR, Adipose PR, Skin Perm, Adipose Perm
Sampled Ranges: 10e3 - 50e3, 1e3 - 25e3, 0.48 - 0.499, 0.48 - 0.499, 10e - 12-10e10, 10e-12 - 10e10
Average parameter loss: [0.08864542 0.17109661 0.08658011 0.1697552  0.07746788 0.10997818]
Average overall loss: 6.543186580971191e-05


([0.28175806094493183,
  0.22326522107635224,
  0.21559711758579528,
  0.21439390230391706,
  0.20980070824069635,
  0.2085879429110459,
  0.20592971997601645,
  0.2035251251821007,
  0.20245677578662122,
  0.19971450551279954,
  0.20105439663997718,
  0.19972823028053557,
  0.19834935824785913,
  0.1968787762203387,
  0.19729679663266456,
  0.19737745209464005,
  0.19490863808563777,
  0.19523732044867106,
  0.19473461327808245,
  0.193350026383996,
  0.19506521044032915,
  0.19160868520183222,
  0.19294776927147592,
  0.19316937827638217,
  0.19308351698730672,
  0.19216688216796943,
  0.19180626049637794,
  0.19012619767870223,
  0.19097346998751163,
  0.19114026986062527,
  0.19089713080653123,
  0.19000161040042127,
  0.1895033536212785,
  0.188910912456257,
  0.18979260618133204,
  0.18880593510610716,
  0.18745762056538037,
  0.18666441392685687,
  0.18547284869211061,
  0.18418091349303722,
  0.18326804201517785,
  0.1833025049418211,
  0.18190629381154264,
  0.1803045403212309

In [181]:
torch.backends.cudnn.benchmark

True

In [7]:
sns.set_theme()

"""
    Plots train and test curves of given models
"""
def printCurves(models, names, train_loss, val_loss, epochs=500):
    fig, ax = plt.subplots(
        len(models)//2, len(models)//2, figsize=(35, len(models)*6), sharex=True, sharey=True, constrained_layout=True)
    x = range(0, epochs)
    fig.supxlabel("Epoch", fontsize=45)
    fig.supylabel("MAE", fontsize=45)

    #ax[0][1].set_xlabel("Training Loss")
    ##ax[1][1].set_xlabel("Validation Loss")
    
    tv_loss = list(zip(train_loss, val_loss))
    for i in range(len(models)//2):
        for j in range(len(models)//2):
            ax[i][j].set_title(names[i+j], fontsize=40)
            ax[i][j].xaxis.set_tick_params(labelsize=35)
            ax[i][j].yaxis.set_tick_params(labelsize=35)
            for k in range(2):
                ax[i][j].plot(x, tv_loss[i+j][k], c = "b" if k == 0 else "y", label = "Train loss" if k == 0 else "Validation loss")
    
    
    """ for i in range(len(tv_loss)):
        for j in range(2):
            print(i//2)
            print(0 if i < len(models)//2 else 1, i//len(models)//2)
            ax[0 if i < len(models)//2 else 1][i//len(models)//2].plot(x, tv_loss[i][j], c = "b" if j == 0 else "y", label = "Train loss" if j == 0 else "Validation loss")"""
    
    #plt.xticks(fontsize=14, rotation=90)
    plt.legend()
    plt.show()